In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from BaselineRemoval import BaselineRemoval
import matplotlib.pyplot as plt
import glob

l = [pd.read_csv(filename,header=None,delimiter = "\t") for 
     filename in glob.glob('C:/Users/perki/OneDrive/Desktop/Machine Learning Demo/H2O2/*.txt')]

data = pd.concat(l, ignore_index=True)
data.columns = ["positionX", "positionY", "Wavenumber", "Counts"]
data.drop(data.columns[[0, 1]], axis = 1, inplace = True)

n =1300
data_separate = pd.DataFrame(np.hstack(np.vsplit(data.values, len(data) // n)))
data_separate.columns = data_separate.columns.map(lambda c: chr(c + ord('A')))

col_to_drop = data_separate.columns[np.array([i for i in range(data_separate.shape[1]) if i != 0 and i%2 != 1])]

rslt_df = data_separate.drop(col_to_drop, axis=1)
   
rslt_df.rename(columns = {'A':'Wavenumber'}, inplace = True)


#Cosmic ray removal    
#function
def modified_z_score(intensity):
 median_int = np.median(intensity)
 mad_int = np.median([np.abs(intensity - median_int)])
 modified_z_scores = 0.6745 * (intensity - median_int) / mad_int
 return modified_z_scores


def fixer(y,m):
 threshold = 5 # binarization threshold. 
 spikes = abs(np.array(modified_z_score(np.diff(y)))) > threshold
 y_out = y.copy() # So we don’t overwrite y
 for i in np.arange(len(spikes)):
  if spikes[i] != 0: # If we have an spike in position i
   w = np.arange(i-m,i+1+m) # we select 2 m + 1 points around our spike
   w2 = w[spikes[w] == 0] # From such interval, we choose the ones which are not spikes
   y_out[i] = np.mean(y[w2])  # and we average their values
 return y_out

# Transform the data to a numpy array
#data1 = rslt_df.drop(columns = ['Wavenumber'])
MASTER = []
for i in rslt_df.columns:
    intensity = rslt_df[i].values.tolist()
    intensity = np.array(intensity)
    fixed_intensity = fixer(intensity,m=7)  
    MASTER.append(fixed_intensity)
    
df = pd.DataFrame(MASTER)
result = df.transpose()
result.columns = result.columns.map(lambda c: chr(c + ord('A')))
#result.to_excel('M1 cosmic ray removal.xlsx', index=False)



#output
new_df = rslt_df['Wavenumber']
a = new_df.to_frame(name='Wavenumber')

frame = pd.concat([a,result], axis = 1)
frame.to_excel('C:/Users/perki/OneDrive/Desktop/Machine Learning Demo/H2O2 + Resveratrol/H2O2 + Res – Cosmic Ray Removal.xlsx', index=False)
